<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/348_WDO_LearningPath_Node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Workforce Development Orchestrator — Learning Path Matching Node

The `learning_path_matching_node` is responsible for converting **prioritized skill gaps** into **concrete, personalized learning recommendations**. This is the point where analysis turns into direct employee impact.

Rather than guessing or optimizing abstract objectives, this node applies **explicit matching rules and thresholds** to ensure that every recommendation is justified, explainable, and achievable.

---

## 1. Strong Preconditions Protect Recommendation Quality

Before attempting any matching, the node verifies that all required inputs are present:

* Employees
* Skill gap analysis
* Available learning paths
* Skill definitions

If any dependency is missing, the node stops immediately and records a clear error.

This prevents a subtle but dangerous failure mode:

> Producing confident learning recommendations without knowing *why* learning is needed.

By enforcing preconditions, the system guarantees that recommendations are always grounded in prior analysis.

---

## 2. Deterministic Matching, Not Probabilistic Guessing

The node delegates all matching logic to the learning path utilities, which are:

* Rules-based
* Configuration-driven
* Independently tested

The node itself contains **no learning logic**, no scoring heuristics, and no thresholds. It simply applies the established policy consistently across the workforce.

This ensures:

* Predictable behavior
* Reproducible results
* Easy auditing

---

## 3. Organization-Wide Consistency

By matching **all employees** in a single pass, the node guarantees:

* Equal treatment across roles and departments
* No special-case logic
* Comparable outcomes across the organization

This is essential for workforce systems that influence training investment and employee experience.

---

## 4. Clean, Focused Output

The node produces a single, well-defined artifact:

* `learning_path_recommendations`

Each recommendation is already enriched with:

* Match confidence
* Priority
* Prerequisite readiness
* Time investment
* Rationale

Downstream nodes and reports can consume this directly without further interpretation.

---

## 5. Errors Are Captured, Not Hidden

Any unexpected failure is recorded in the shared error log rather than terminating execution.

This allows:

* Partial insights to still be delivered
* Failures to be diagnosed post-run
* Trust to be maintained even when inputs change

This is especially important in long, multi-stage orchestrations.

---

## Why This Node Builds Trust

From a leadership perspective, this node ensures:

* Learning spend is directly tied to documented skill gaps
* Recommendations are policy-driven, not model-driven
* Employees are not overwhelmed with low-value suggestions

From an employee perspective, it ensures:

* Learning recommendations are relevant
* Prerequisites are respected
* The “why” behind each recommendation is clear

---

## Architectural Takeaway

This node reinforces a key principle of responsible AI systems:

> **Learning recommendations should be the consequence of analysis, not the output of a model.**

By strictly sequencing gap detection → prioritization → learning matching, the orchestrator produces development plans that are defensible, fair, and aligned with real workforce risk.



In [ ]:
def learning_path_matching_node(
    state: WorkforceDevelopmentOrchestratorState,
    config: WorkforceDevelopmentOrchestratorConfig
) -> Dict[str, Any]:
    """
    Learning Path Matching Node: Orchestrate matching employees to learning paths.

    Matches employees to appropriate learning paths based on their skill gaps.
    """
    errors = state.get("errors", [])

    # Get required data from state
    employees = state.get("employees", [])
    skill_gap_analysis = state.get("skill_gap_analysis", [])
    learning_paths = state.get("learning_paths", [])
    skills_lookup = state.get("skills_lookup", {})

    if not employees:
        return {
            "errors": errors + ["learning_path_matching_node: employees data required"]
        }

    if not skill_gap_analysis:
        return {
            "errors": errors + ["learning_path_matching_node: skill_gap_analysis data required"]
        }

    if not learning_paths:
        return {
            "errors": errors + ["learning_path_matching_node: learning_paths data required"]
        }

    if not skills_lookup:
        return {
            "errors": errors + ["learning_path_matching_node: skills_lookup data required"]
        }

    try:
        # Match all employees to learning paths
        recommendations = match_all_employees_to_learning_paths(
            employees,
            skill_gap_analysis,
            learning_paths,
            skills_lookup,
            config
        )

        return {
            "learning_path_recommendations": recommendations,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"learning_path_matching_node: Unexpected error: {str(e)}"]
        }



# Test learning path matching node

In [ ]:
"""Test learning path matching node

Testing Phase 5: Learning Path Matching Node
Following the pattern: Test node after utilities pass
"""

from agents.workforce_development_orchestrator.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    automation_risk_analysis_node,
    skill_gap_detection_node,
    learning_path_matching_node
)
from config import (
    WorkforceDevelopmentOrchestratorState,
    WorkforceDevelopmentOrchestratorConfig
)


def test_learning_path_matching_node():
    """Test learning path matching node"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # Load data and run previous nodes
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    risk_update = automation_risk_analysis_node(state, config)
    state.update(risk_update)

    gap_update = skill_gap_detection_node(state, config)
    state.update(gap_update)

    # Then match learning paths
    result = learning_path_matching_node(state, config)

    # Check that recommendations are present
    assert "learning_path_recommendations" in result
    assert len(result["learning_path_recommendations"]) > 0

    # Check structure of first recommendation
    first_rec = result["learning_path_recommendations"][0]
    assert "employee_id" in first_rec
    assert "learning_path_id" in first_rec
    assert "match_score" in first_rec
    assert "target_skill" in first_rec
    assert "prerequisites_met" in first_rec

    # Verify match scores meet threshold
    assert all(
        r["match_score"] >= config.learning_path_match_threshold
        for r in result["learning_path_recommendations"]
    )

    # Verify no errors
    assert len(result.get("errors", [])) == 0

    print("✅ test_learning_path_matching_node: PASSED")


def test_learning_path_matching_node_requires_data():
    """Test learning path matching node requires data"""
    state: WorkforceDevelopmentOrchestratorState = {
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    result = learning_path_matching_node(state, config)

    # Should have errors
    assert "errors" in result
    assert len(result["errors"]) > 0

    print("✅ test_learning_path_matching_node_requires_data: PASSED")


def test_learning_path_matching_integration():
    """Test learning path matching with full workflow"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }
    config = WorkforceDevelopmentOrchestratorConfig()

    # Full workflow up to learning path matching
    goal_update = goal_node(state)
    state.update(goal_update)

    planning_update = planning_node(state)
    state.update(planning_update)

    data_update = data_loading_node(state, config)
    state.update(data_update)

    risk_update = automation_risk_analysis_node(state, config)
    state.update(risk_update)

    gap_update = skill_gap_detection_node(state, config)
    state.update(gap_update)

    path_update = learning_path_matching_node(state, config)
    state.update(path_update)

    # Verify all data is present
    assert "employees" in state
    assert "skill_gap_analysis" in state
    assert "learning_path_recommendations" in state
    assert len(state["learning_path_recommendations"]) > 0

    # Verify recommendation quality
    for rec in state["learning_path_recommendations"]:
        assert 0.0 <= rec["match_score"] <= 1.0
        assert rec["match_score"] >= config.learning_path_match_threshold
        assert "employee_id" in rec
        assert "learning_path_id" in rec

    print("✅ test_learning_path_matching_integration: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Learning Path Matching Node (Phase 5)")
    print("=" * 60)
    print()

    test_learning_path_matching_node()
    test_learning_path_matching_node_requires_data()
    test_learning_path_matching_integration()

    print()
    print("=" * 60)
    print("✅ All learning path node tests passed!")
    print("=" * 60)



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_learning_path_node.py
============================================================
Testing Learning Path Matching Node (Phase 5)
============================================================

✅ test_learning_path_matching_node: PASSED
✅ test_learning_path_matching_node_requires_data: PASSED
✅ test_learning_path_matching_integration: PASSED

============================================================
✅ All learning path node tests passed!
============================================================
